In [2]:
import json

with open('ini/mysql_local.json', 'r') as file:
 config_str = file.read()

config = json.loads(config_str)

import pymysql

conn = pymysql.connect(
 host = config['host'],
 user = config['user'],
 password = config['password'],
 database = config['database'],
 port = config['port']
)

## 패스워드 암호화
    - Secure Hash Algorithm(SHA-256)

In [1]:
import hashlib

pwd = "1234"
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- base64로 인코딩

In [3]:
import base64

base64.b16encode(pwd_sha256.digest())

b'03AC674216F3E15C761EE1A5E255F067953623C8B388B4459E13F978D7C846F4'

In [4]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [5]:
len(hashed_pwd)

44

- 패스워드를 생성하는 함수

In [6]:
import hashlib, base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [7]:
gen_pwd("qwerty")

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기
    - uid + 21

In [8]:
sql = "select uid from users;"
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('cpg',),
 ('djy',),
 ('dskim',),
 ('gdhong',),
 ('ils',),
 ('soy',),
 ('wjlee',))

In [12]:
sql_update = 'update users set pwd = %s where uid = %s;'

for line in results:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update, (pwd, uid))

conn.commit()

- email 추가 : uid@mc.com

In [13]:
sql_update = 'update users set email = %s where uid = %s;'

for line in results:
    uid = line[0]
    email = uid + '@mc.com'
    cur.execute(sql_update, (email, uid))

conn.commit()

## 로그인

In [37]:
sql_login = 'select uid, pwd from users where uid = %s and is_deleted = 0'

- case 1) uid가 없는경우

In [38]:
uid = 'fool'
cur.execute(sql_login, (uid, ))
result = cur.fetchone()
result

- case 2) 비밀번호가 틀린경우

In [39]:
uid, pwd = 'djy','djy123'
cur.execute(sql_login, (uid, ))
result = cur.fetchone()
result

In [40]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!!')

TypeError: 'NoneType' object is not subscriptable

- case 3) OK

In [23]:
uid, pwd = 'djy','djy123'
cur.execute(sql_login, (uid, ))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!!')

TypeError: 'NoneType' object is not subscriptable

In [42]:
uid, pwd = 'djy','djy123'
cur.execute(sql_login, (uid, ))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!!')
else:
    print('bad uid!!!')

bad uid!!!


In [41]:
uid, pwd = 'djy','djy123'
cur.execute(sql_login, (uid, ))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!!')
except:
    print('bad uid!!!')

bad uid!!!


In [43]:
cur.close()
conn.close()